### 1. セットアップ
時間がかかる場合がありますが、ブラウザを閉じずにお待ちください。

In [ ]:
# Node.jsのインストール
!dpkg --configure -a > /dev/null
!sudo apt-get update > /dev/null
!sudo apt-get install -y ca-certificates curl gnupg > /dev/null
!sudo mkdir -p /etc/apt/keyrings
!curl -fsSL https://deb.nodesource.com/gpgkey/nodesource-repo.gpg.key | sudo gpg --dearmor -o /etc/apt/keyrings/nodesource.gpg
!NODE_MAJOR=18 && echo "deb [signed-by=/etc/apt/keyrings/nodesource.gpg] https://deb.nodesource.com/node_$NODE_MAJOR.x nodistro main" | sudo tee /etc/apt/sources.list.d/nodesource.list
!sudo apt-get update > /dev/null
!sudo apt-get install nodejs -y > /dev/null

# 必要なライブラリのインストール
!pip install jinja2 > /dev/null
!pip install openpyxl > /dev/null
!pip install pandas > /dev/null

from datetime import datetime
import json
import os
import re
from urllib.parse import urlparse

from google.colab import files
from jinja2 import Template, FileSystemLoader, Environment
import openpyxl
import pandas as pd

###### 関数 ######

def get_datetime():
    return datetime.now().strftime('%Y%m%d%H%M%S')

##################

###### 定数 ######

env = Environment(loader=FileSystemLoader('.'))

##################

!rm -rf /content/dakit2-minimal
!git clone https://github.com/adachi-a/dakit2-minimal
os.chdir("/content/dakit2-minimal")
!npm install > /dev/null

In [ ]:
# @title 2. 設定
サイトのタイトル = "" # @param {type:"string"}
dakit_site_title = サイトのタイトル
サイトの作成者 = "" # @param {type:"string"}
dakit_site_author = サイトの作成者
サイトのURL = "" # @param {type:"string"}
dakit_site_url = サイトのURL
検索エンジンにインデックスさせない = True # @param {type:"boolean"}
dakit_noindex = 検索エンジンにインデックスさせない
カラーテーマ = "success" # @param {type:"string"}
dakit_bs_theme_bg = カラーテーマ

if dakit_site_title == "":
    raise ValueError("dakit_site_titleを空白にすることはできません。")
if dakit_site_author == "":
    raise ValueError("dakit_site_authorを空白にすることはできません。")
if dakit_site_url == "":
    raise ValueError("dakit_site_urlを空白にすることはできません。")

if not dakit_site_url.endswith("/"):
    dakit_site_url += "/" # add trailing slash

base_url_parse_result = urlparse(dakit_site_url)

# vite.config.js
with open("./vite.config.js", "w") as f:
    f.write(f"""
import {{ defineConfig }} from 'vite';
import {{ svelte }} from '@sveltejs/vite-plugin-svelte';

// https://vitejs.dev/config/
export default defineConfig({{
  plugins: [svelte()],
  base: '{base_url_parse_result.path}',
}});
    """)

 ### 3. メタデータの読み込み
 - 読み込ませるExcelブックは、メタデータを記載した`data`と画像情報を記載した`images`の2つのシートだけを持つ必要があります。
 - `data`シートの一番左の列は`search_id`とし、列内で重複のないようにします（検索用であり資料のメタデータとは異なります）。
 - `images`シートは`id`列と`path`列を持ち、`id`列は`data`シートと対応するようにします。`path`列には画像のパスまたはURLを記述します。

In [ ]:
uploaded = files.upload()
metadata_filename = next(iter(uploaded))
book = openpyxl.load_workbook(metadata_filename)

if book.sheetnames != ["data", "images"]:
    raise ValueError('Excelブックは"data"と”images"の2つのシートだけを持つようにします。')

catalog = pd.read_excel(metadata_filename, sheet_name="data").fillna("")
image_list = pd.read_excel(metadata_filename, sheet_name="images").fillna("")

if catalog.columns[0] != "search_id":
    raise ValueError('dataの一番左の列は常に"search_id"とする必要があります。')

os.makedirs("./public", exist_ok=True)
catalog.to_json("./public/data.json", orient="records", force_ascii=False)

# catalogの列一覧を取得し、JDCatメタデータスキーマとのマッピング用のExcelファイルを返す
catalog_columns = pd.DataFrame({"列名": catalog.columns, "基本項目": ["" for _ in range(len(catalog.columns))], "検索ページに表示": ["" for _ in range(
    len(catalog.columns))], "絞り込み": ["" for _ in range(len(catalog.columns))], "ソート": ["" for _ in range(len(catalog.columns))], "検索対象": ["" for _ in range(len(catalog.columns))]}).T
catalog_columns_datetime = get_datetime()
catalog_columns.to_excel(f"setting_{catalog_columns_datetime}.xlsx", index=True)
files.download(f"setting_{catalog_columns_datetime}.xlsx")

### 4. 設定ファイルの読み込み
- 下のセルを実行して、設定用のExcelファイルを読み込みます。

In [ ]:
uploaded = files.upload()
setting_filename = next(iter(uploaded))

setting_df = pd.read_excel(setting_filename, header=0, index_col=None, dtype=str).fillna("").T

if setting_df[1].str.contains("ID").sum() != 1:
    raise ValueError("IDは1つだけ指定してください。")
else:
    id_row = tuple(setting_df[setting_df[1].str.contains("ID")][0])[0]

if setting_df[1].str.contains("Title").sum() != 1:
    raise ValueError("Titleは1つだけ指定してください。")
else:
    title_row = tuple(setting_df[setting_df[1].str.contains("Title")][0])[0]

if setting_df[2].apply(lambda x: len(x.split(",")) if x != "" else 0).sum() <= 1:
    raise ValueError("検索ページに表示する項目は1つ以上指定してください。")

if setting_df[3].apply(lambda x: len(x.split(",")) if x != "" else 0).sum() <= 1:
    raise ValueError("絞り込み項目は1つ以上指定してください。")

if setting_df[4].apply(lambda x: len(x.split(",")) if x != "" else 0).sum() <= 1:
    raise ValueError("ソート項目は1つ以上指定してください。")

if setting_df[5].apply(lambda x: len(x.split(",")) if x != "" else 0).sum() <= 1:
    raise ValueError("検索対象項目は1つ以上指定してください。")

### 5. ページの生成
- 下のセルを実行すると、ページが生成されます。

In [ ]:
display_rows = setting_df[setting_df[2] == "1"][0].values
filter_rows = setting_df[setting_df[3] == "1"][0].values
sort_rows = setting_df[setting_df[4] == "1"][0].values
search_target_rows = setting_df[setting_df[5] == "1"][0].values

dakit_item_id_column_name = id_row
dakit_item_title_column_name = title_row

fuse_keys = list(search_target_rows)

# ソートの設定
itemsjs_sortings = {}
for row in sort_rows:
    itemsjs_sortings[f"{row}(昇順)"] = {"field": row, "order": "asc"}
    itemsjs_sortings[f"{row}(降順)"] = {"field": row, "order": "desc"}

# フィルタの設定
itemsjs_aggregations = {}
for row in filter_rows:
    itemsjs_aggregations[row] = {"title": row, "conjunction": False}

# 検索ページの表示方法の設定
dakit_display_keys = list(display_rows)
dakit_display_keys_string = ",".join(dakit_display_keys)

dakit_display_items_string = ""

for key in dakit_display_keys:
    dakit_display_items_string += f"{key}: {{{key}}} / "

dakit_display_items_string = dakit_display_items_string[:-3]

dakit_display_template_string = f"""
<a href="./items/{{{dakit_item_id_column_name}}}.html" target="_blank"><h5 class="mb-0">{{{dakit_item_title_column_name}}}</h5></a>
<p class="mb-1"><small>{dakit_display_items_string}</small></p>
"""

# dakit_fuse_keys.json
with open("./src/dakit_fuse_keys.json", "w") as f:
    json.dump(fuse_keys, f, indent=2, ensure_ascii=False)

# dakit_itemsjs_configuration.json
itemsjs_configuration: dict = {
    "sortings": itemsjs_sortings,
    "aggregations": itemsjs_aggregations,
    "removeStopWordFilter": True,
    "native_search_enabled": False,
    "custom_id_field": 'search_id'
}

with open("./src/dakit_itemsjs_configuration.json", "w") as f:
    json.dump(itemsjs_configuration, f, indent=2, ensure_ascii=False)

template = env.get_template('./kit/Search.svelte.jinja')

with open("./src/Search.svelte", "w") as f:
    f.write(template.render(
        dakit_site_title=dakit_site_title,
        dakit_site_url=dakit_site_url,
        dakit_noindex=dakit_noindex,
        dakit_display_keys_string=dakit_display_keys_string,
        dakit_display_template_string=dakit_display_template_string,
    ))

template = env.get_template('./kit/index.html.jinja')

with open("./index.html", "w") as f:
    f.write(template.render(
        dakit_site_title=dakit_site_title
    ))

template = env.get_template('./kit/App.svelte.jinja')

with open("./src/App.svelte", "w") as f:
    f.write(template.render(
        dakit_site_title=dakit_site_title,
        dakit_noindex=dakit_noindex,
        dakit_bs_theme_bg=dakit_bs_theme_bg
    ))

!npm run build

template_image = env.get_template('./kit/item.html.jinja')
template_no_image = env.get_template('./kit/item_no_image.html.jinja')
items_with_image = set(image_list["id"].to_list())
os.makedirs("./dist/items/images", exist_ok=True)

for item in catalog.iterrows():
    item_id = item[1][dakit_item_id_column_name]
    item_title = item[1][dakit_item_title_column_name]
    item_metadata = item[1].to_dict()
    if item_id not in items_with_image:
        template = template_no_image
        with open(f"./dist/items/{item_id}.html", "w") as f:
            f.write(template.render(
                dakit_site_title=dakit_site_title,
                dakit_site_url=dakit_site_url,
                dakit_noindex=dakit_noindex,
                dakit_bs_theme_bg=dakit_bs_theme_bg,
                item_title=item_title,
                item_metadata=item_metadata
            ))
    else:
        template = template_image
        item_image_list = image_list[image_list["id"] == item_id]["path"].to_list()
        item_image_array = [{"src": x} for x in item_image_list]
        with open(f"./dist/items/{item_id}.html", "w") as f:
            f.write(template.render(
                dakit_site_title=dakit_site_title,
                dakit_site_url=dakit_site_url,
                dakit_noindex=dakit_noindex,
                dakit_bs_theme_bg=dakit_bs_theme_bg,
                item_title=item_title,
                item_metadata=item_metadata,
                item_image_list=item_image_list,
                item_image_array=item_image_array
            ))

!npx html-beautify --preserve-newlines false ./dist/*/*.html
!touch ./dist/.nojekyll
!zip -r /content/dakit2_output.zip /content/dakit2-minimal/dist
files.download("/content/dakit2_output.zip")
print("生成が完了しました。")

ダウンロードされたzipファイルをサーバ上に展開し、必要に応じて画像ファイルを追加したら完成です。